### Preferences model

In [1]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from network_functions import calculate_network_utilities, new_screening_strategy, old_screening_strategy, create_folders_logger
from simulations import plot_classification_results
from plots import plot_estimations_w_error_bars, plot_screening_counts

from preprocessing import preprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt

import yaml
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

import argparse
import pdb
import matplotlib
matplotlib.use('Agg')

import logging
import datetime 
import os

In [2]:
cfg['max_workers']

4

In [3]:
cfg['rel_point_cond_mut_info']

{'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}

In [4]:
# from main import main
from use_case_new_strategy import use_case_new_strategy
from full_example import full_example
from update import update_influence_diagram
from network_functions import create_folders_logger
from sens_analysis_elicitation import sens_analysis_elicitation
from sens_analysis_PE_method import sens_analysis_PE_method

In [5]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

## Use cases

### Individual cases

**Benchmark patient**

In [6]:
label = 'notebook_benchmark_patient'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-27 13:07:52,600 - INFO - Model type: linear
2026-01-27 13:07:52,600 - INFO - Reading network...
2026-01-27 13:07:52,625 - INFO - Calculating information values...
2026-01-27 13:08:17,563 - INFO - Eliciting value of comfort...
2026-01-27 13:08:17,566 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:08:17,567 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:08:17,569 - INFO - Adding values for 2 new tests...
2026-01-27 13:09:04,071 - INFO - Saving network...
2026-01-27 13:09:04,155 - INFO - Plotting info functions...
2026-01-27 13:09:09,171 - INFO - Calculating final utilities...
2026-01-27 13:09:09,186 - INFO - Best info: 15.636459410190582
2026-01-27 13:09:09,188 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 13:09:09,188 - INFO - PE info: 4.1
2026-01-27 13:09:09,188 - INFO - v_best: 106.35895

**Benchmark patient risk seeking**

In [7]:
label = 'notebook_benchmark_risk_seeking'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    change_risk_param=True,
    rho_param= 0.005
)

2026-01-27 13:09:09,428 - INFO - Model type: linear
2026-01-27 13:09:09,428 - INFO - Model type: linear
2026-01-27 13:09:09,428 - INFO - Reading network...
2026-01-27 13:09:09,428 - INFO - Reading network...
2026-01-27 13:09:09,441 - INFO - Calculating information values...
2026-01-27 13:09:09,441 - INFO - Calculating information values...
2026-01-27 13:09:35,424 - INFO - Eliciting value of comfort...
2026-01-27 13:09:35,424 - INFO - Eliciting value of comfort...
2026-01-27 13:09:35,426 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:09:35,426 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:09:35,426 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:09:35,426 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.8019

**Benchmark patient added evidence**

In [8]:
label = 'notebook_benchmark_addded_ev'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = cfg["patient_chars"].copy()
patient_chars["Diabetes"] = True
patient_chars["Hypertension"] = True

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
)

2026-01-27 13:10:22,237 - INFO - Model type: linear
2026-01-27 13:10:22,237 - INFO - Reading network...
2026-01-27 13:10:22,263 - INFO - Calculating information values...
2026-01-27 13:10:46,809 - INFO - Eliciting value of comfort...
2026-01-27 13:10:46,809 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:10:46,809 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:10:46,813 - INFO - Adding values for 2 new tests...
2026-01-27 13:11:27,073 - INFO - Saving network...
2026-01-27 13:11:27,147 - INFO - Plotting info functions...
2026-01-27 13:11:32,853 - INFO - Calculating final utilities...
2026-01-27 13:11:32,866 - INFO - Best info: 15.636459410190582
2026-01-27 13:11:32,868 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 13:11:32,869 - INFO - PE info: 4.1
2026-01-27 13:11:32,869 - INFO - v_best: 106.35895

**Exogenous variable p = 0.02**

In [9]:
label = 'notebook_exogenous_var_002'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }


update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.02
)

2026-01-27 13:11:33,122 - INFO - Model type: linear
2026-01-27 13:11:33,124 - INFO - Reading network...
2026-01-27 13:11:33,141 - INFO - Calculating information values...
2026-01-27 13:11:57,650 - INFO - Eliciting value of comfort...
2026-01-27 13:11:57,650 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:11:57,650 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:11:57,665 - INFO - Adding values for 2 new tests...
2026-01-27 13:12:34,347 - INFO - Saving network...
2026-01-27 13:12:34,409 - INFO - Plotting info functions...
2026-01-27 13:12:39,376 - INFO - Calculating final utilities...
2026-01-27 13:12:39,378 - INFO - Best info: 15.636459410190582
2026-01-27 13:12:39,378 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 13:12:39,378 - INFO - PE info: 4.1
2026-01-27 13:12:39,378 - INFO - v_best: 106.35895

**Exogenous variable p = 0.1**

In [10]:
label = 'notebook_exogenous_var_01'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.1
)

2026-01-27 13:12:39,543 - INFO - Model type: linear
2026-01-27 13:12:39,545 - INFO - Reading network...
2026-01-27 13:12:39,561 - INFO - Calculating information values...
2026-01-27 13:13:03,824 - INFO - Eliciting value of comfort...
2026-01-27 13:13:03,824 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:13:03,824 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:13:03,824 - INFO - Adding values for 2 new tests...
2026-01-27 13:13:39,602 - INFO - Saving network...
2026-01-27 13:13:39,680 - INFO - Plotting info functions...
2026-01-27 13:13:45,338 - INFO - Calculating final utilities...
2026-01-27 13:13:45,351 - INFO - Best info: 15.636459410190582
2026-01-27 13:13:45,353 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 13:13:45,355 - INFO - PE info: 4.1
2026-01-27 13:13:45,356 - INFO - v_best: 106.35895

### Assessing the French strategy

**Patient 1**

In [11]:
label = 'notebook_4_2_patient_1'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_5_old_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": False,
    "Hypertension": False,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-27 13:13:45,603 - INFO - Model type: linear
2026-01-27 13:13:45,603 - INFO - Reading network...
2026-01-27 13:13:45,627 - INFO - Calculating information values...
2026-01-27 13:14:09,684 - INFO - Eliciting value of comfort...
2026-01-27 13:14:09,684 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:14:09,684 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:14:09,684 - INFO - Adding values for 2 new tests...
2026-01-27 13:14:45,550 - INFO - Saving network...
2026-01-27 13:14:45,634 - INFO - Plotting info functions...
2026-01-27 13:14:50,545 - INFO - Calculating final utilities...
2026-01-27 13:14:50,554 - INFO - Best info: 15.636459410190582
2026-01-27 13:14:50,554 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 13:14:50,556 - INFO - PE info: 4.1
2026-01-27 13:14:50,557 - INFO - v_best: 106.35895

**Patient 2**

In [12]:
label = 'notebook_4_2_patient_2'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": True,
    "Hypertension": True,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-27 13:14:50,714 - INFO - Model type: linear
2026-01-27 13:14:50,719 - INFO - Reading network...
2026-01-27 13:14:50,738 - INFO - Calculating information values...
2026-01-27 13:15:17,666 - INFO - Eliciting value of comfort...
2026-01-27 13:15:17,669 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:15:17,671 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:15:17,672 - INFO - Adding values for 2 new tests...
2026-01-27 13:15:53,571 - INFO - Saving network...
2026-01-27 13:15:53,656 - INFO - Plotting info functions...
2026-01-27 13:15:58,641 - INFO - Calculating final utilities...
2026-01-27 13:15:58,648 - INFO - Best info: 15.636459410190582
2026-01-27 13:15:58,650 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 13:15:58,650 - INFO - PE info: 4.1
2026-01-27 13:15:58,651 - INFO - v_best: 106.35895

**Patient 3**

In [13]:
label = 'notebook_4_2_patient_3'
logger, log_dir = create_folders_logger(label = label)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_1",
    "Smoking": "sm_3_ex_smoker",
    "BMI": "bmi_3_overweight",
    "Alcohol": "high",
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-27 13:15:58,893 - INFO - Model type: linear
2026-01-27 13:15:58,893 - INFO - Reading network...
2026-01-27 13:15:58,917 - INFO - Calculating information values...
2026-01-27 13:16:24,201 - INFO - Eliciting value of comfort...
2026-01-27 13:16:24,201 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 13:16:24,204 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 13:16:24,207 - INFO - Adding values for 2 new tests...
2026-01-27 13:17:08,406 - INFO - Saving network...
2026-01-27 13:17:08,485 - INFO - Plotting info functions...
2026-01-27 13:17:13,670 - INFO - Calculating final utilities...
2026-01-27 13:17:13,675 - INFO - Best info: 15.636459410190582
2026-01-27 13:17:13,675 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 13:17:13,675 - INFO - PE info: 4.1
2026-01-27 13:17:13,683 - INFO - v_best: 106.35895

### Designing a national personalised strategy

In [14]:
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

label = 'notebook_personalised_screening_str'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    single_run=cfg['single_run'],
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = ''
)

2026-01-27 13:17:13,907 - INFO - Configuration variables of interest:
2026-01-27 13:17:13,909 - INFO - Single run: True
2026-01-27 13:17:13,910 - INFO - Use all variables: True
2026-01-27 13:17:13,912 - INFO - Use case with new test: True
2026-01-27 13:17:13,912 - INFO - Reading the network file...
2026-01-27 13:17:13,933 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestTrue/decision_models/DM_screening_rel_point_cond_mut_info_linear_new_test.xdsl
2026-01-27 13:17:13,934 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 7.0, 3 - 6.801984517973132, 4 - 7.0
2026-01-27 13:17:14,645 - INFO - All variables are kept in the dataframe for calculation of utilities.
2026-01-27 13:17:14,645 - WARNING - Limit for New_Test_A not found in config, setting to default (20000).
2026-01-27 13:17:14,645 - WARNING - Limit for New_Test_B not found in config, setting to default (20000).
2026-01-27 13:17:14,650 - INFO - Operational limits for the screening strategies: {'No_scr_

### Benchmarking of new screening devices

In [6]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

In [7]:
label = 'notebook_benchmark_new_screening'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = True,
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-02-04 09:58:22,920 - INFO - Model type: linear
2026-02-04 09:58:22,921 - INFO - Reading network...
2026-02-04 09:58:22,974 - INFO - Calculating information values...
2026-02-04 09:58:51,028 - INFO - Eliciting value of comfort...
2026-02-04 09:58:51,029 - INFO - No elicitation of lambda values, taking default values...
2026-02-04 09:58:51,034 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-02-04 09:58:51,035 - INFO - Adding values for 2 new tests...
2026-02-04 09:59:29,017 - INFO - Saving network...
2026-02-04 09:59:29,080 - INFO - Plotting info functions...
2026-02-04 09:59:35,274 - INFO - Calculating final utilities...
2026-02-04 09:59:35,277 - INFO - Best info: 15.636459410190582
2026-02-04 09:59:35,277 - INFO - Worst info: -1.4901161193847656e-08
2026-02-04 09:59:35,277 - INFO - PE info: 4.1
2026-02-04 09:59:35,277 - INFO - v_best: 106.35895

In [8]:
cfg["lambda_list"]

[4.011942483017116, 4.168480067521723, 6.801984517973132, 7.0]

In [9]:
label = 'notebook_new_test_screening_strategy'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    use_case_new_test = True,
    single_run=cfg['single_run'],
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = '',
    full_analysis= False
)

2026-02-04 09:59:35,553 - INFO - Configuration variables of interest:
2026-02-04 09:59:35,553 - INFO - Configuration variables of interest:
2026-02-04 09:59:35,556 - INFO - Single run: True
2026-02-04 09:59:35,556 - INFO - Single run: True
2026-02-04 09:59:35,557 - INFO - Use all variables: True
2026-02-04 09:59:35,557 - INFO - Use all variables: True
2026-02-04 09:59:35,558 - INFO - Use case with new test: True
2026-02-04 09:59:35,558 - INFO - Use case with new test: True
2026-02-04 09:59:35,559 - INFO - Reading the network file...
2026-02-04 09:59:35,559 - INFO - Reading the network file...
2026-02-04 09:59:35,613 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestTrue/decision_models/DM_screening_rel_point_cond_mut_info_linear_new_test.xdsl
2026-02-04 09:59:35,613 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestTrue/decision_models/DM_screening_rel_point_cond_mut_info_linear_new_test.xdsl
2026-02-04 09:59:35,613 - INFO - Com

## Sensitivity analysis

### Sensitivity analysis for the PE

In [5]:
label = 'notebook_sens_analysis_PE_method'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

sens_analysis_PE_method(
    label = label,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,  
)

2025-01-27 17:50:51,100 - INFO - Reading from file: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-27 17:50:51,100 - INFO - Reading from file: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-27 17:50:51,102 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-27 17:50:51,102 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-27 17:50:51,103 - INFO - PE_cost_array: [ 5 10 50]
2025-01-27 17:50:51,103 - INFO - PE_cost_array: [ 5 10 50]
2025-01-27 17:50:54,889 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-27 17:50:54,889 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-27 17:51:59,901 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-27 17:51:59,901 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-27 17:52:00,603 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.036469730286183666]
2025-01-27 17:52:00,603 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.0364

### Sensitivity analysis for comfort parameters

In [6]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

sens_analysis_elicitation(
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:04:17,390 - INFO - Configuration variables of interest:
2025-01-29 13:04:17,390 - INFO - Configuration variables of interest:
2025-01-29 13:04:17,391 - INFO - Single run: False
2025-01-29 13:04:17,391 - INFO - Single run: False
2025-01-29 13:04:17,393 - INFO - Use all variables: True
2025-01-29 13:04:17,393 - INFO - Use all variables: True
2025-01-29 13:04:17,394 - INFO - Use case with new test: False
2025-01-29 13:04:17,394 - INFO - Use case with new test: False
2025-01-29 13:04:17,395 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:04:17,395 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:04:17,396 - INFO - Change lambdas? True
2025-01-29 13:04:17,396 - INFO - Change lambdas? True
2025-01-29 13:04:17,397 - INFO - Read lambda list from config? True
2025-01-29 13:04:17,397 - INFO - Read lambda list from config? True
2025-01-29 13:04:17,398 - INFO - Lambdas: [4.011942483017116, 4.168480067521723, 6.8019845

Counts: best_option
No_scr_no_col    288682
No_scr_col            0
gFOBT                 0
FIT                3201
Blood_based           0
Stool_DNA         47824
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [7]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"].copy()
lambda_list[2] = 6.3

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2

Counts: best_option
No_scr_no_col    313960
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA         25747
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [8]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"]
lambda_list[0] = 4.8
lambda_list[1] = 5

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:10:02,223 - INFO - Configuration variables of interest:
2025-01-29 13:10:02,223 - INFO - Configuration variables of interest:
2025-01-29 13:10:02,226 - INFO - Single run: False
2025-01-29 13:10:02,226 - INFO - Single run: False
2025-01-29 13:10:02,228 - INFO - Use all variables: True
2025-01-29 13:10:02,228 - INFO - Use all variables: True
2025-01-29 13:10:02,230 - INFO - Use case with new test: False
2025-01-29 13:10:02,230 - INFO - Use case with new test: False
2025-01-29 13:10:02,231 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:10:02,231 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:10:02,233 - INFO - Change lambdas? True
2025-01-29 13:10:02,233 - INFO - Change lambdas? True
2025-01-29 13:10:02,234 - INFO - Read lambda list from config? True
2025-01-29 13:10:02,234 - INFO - Read lambda list from config? True
2025-01-29 13:10:02,235 - INFO - Lambdas: [4.8, 5, 6.801984517973132, 7.0]
2025-01-29 13:1

Counts: best_option
No_scr_no_col    179840
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA        159867
CTC                   0
Colon_capsule         0
Name: count, dtype: int64
